In [2]:
import pandas as pd
import sqlite3

In [7]:
db_path = "../data/traffic_data.db"
conn=sqlite3.connect(db_path)
df=pd.read_sql_query('select * from traffic_data',conn)

print("Data Shape:", df.shape)
print(df.head())

Data Shape: (1146, 9)
   id location_name      lat      lon                   timestamp  \
0   1         Dadar  19.0176  72.8562  2025-07-23T23:25:24.659867   
1   2       Andheri  19.1197  72.8468  2025-07-23T23:25:25.672796   
2   3         Powai  19.1197  72.9051  2025-07-23T23:25:26.528793   
3   4        Colaba  18.9067  72.8147  2025-07-23T23:25:27.154680   
4   5        Panvel  18.9894  73.1175  2025-07-23T23:25:27.893369   

   current_speed  free_flow_speed  confidence road_class  
0           27.0             27.0         1.0    Unknown  
1           22.0             22.0         1.0    Unknown  
2           31.0             31.0         1.0    Unknown  
3           23.0             23.0         1.0    Unknown  
4           21.0             21.0         1.0    Unknown  


In [4]:
import sqlite3
import pandas as pd

db_path = "../data/traffic_data.db"
conn = sqlite3.connect(db_path)

df = pd.read_sql_query("SELECT * FROM traffic_data_enriched", conn)
conn.close()

print("Shape:", df.shape)
print(df.head())


Shape: (1146, 14)
   id location_name      lat      lon                   timestamp  \
0   1         Dadar  19.0176  72.8562  2025-07-23 23:25:24.659867   
1   2       Andheri  19.1197  72.8468  2025-07-23 23:25:25.672796   
2   3         Powai  19.1197  72.9051  2025-07-23 23:25:26.528793   
3   4        Colaba  18.9067  72.8147  2025-07-23 23:25:27.154680   
4   5        Panvel  18.9894  73.1175  2025-07-23 23:25:27.893369   

   current_speed  free_flow_speed  confidence road_class  is_weekend  \
0           27.0             27.0         1.0    Unknown           0   
1           22.0             22.0         1.0    Unknown           0   
2           31.0             31.0         1.0    Unknown           0   
3           23.0             23.0         1.0    Unknown           0   
4           21.0             21.0         1.0    Unknown           0   

   is_rush_hour  congestion_level  traffic_ratio  speed_drop_percent  
0             0               0.0            1.0               

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ➤ Add only required derived columns
df['is_weekend'] = df['timestamp'].dt.weekday.isin([5,6]).astype(int)
df['is_rush_hour'] = df['timestamp'].dt.hour.isin([7,8,9,17,18,19]).astype(int)
df['congestion_level'] = df['free_flow_speed'] - df['current_speed']
df['traffic_ratio'] = df['current_speed'] / df['free_flow_speed']
df['speed_drop_percent'] = (1 - (df['current_speed'] / df['free_flow_speed'])) * 100


# Replace infinities & NaNs in ratio
df['traffic_ratio'] = df['traffic_ratio'].replace([float('inf'), -float('inf')], None)

# ➤ Save as enriched table
df.to_sql("traffic_data_enriched", conn, if_exists="replace", index=False)

print("✅ Enriched data saved to table 'traffic_data_enriched'")

✅ Enriched data saved to table 'traffic_data_enriched'


In [4]:
import sqlite3
import pandas as pd

db_path = "../data/traffic_data.db"
conn = sqlite3.connect(db_path)

df = pd.read_sql_query("SELECT * FROM traffic_data_enriched", conn)
conn.close()

print("Shape:", df.shape)
print(df.head())


Shape: (1146, 14)
   id location_name      lat      lon                   timestamp  \
0   1         Dadar  19.0176  72.8562  2025-07-23 23:25:24.659867   
1   2       Andheri  19.1197  72.8468  2025-07-23 23:25:25.672796   
2   3         Powai  19.1197  72.9051  2025-07-23 23:25:26.528793   
3   4        Colaba  18.9067  72.8147  2025-07-23 23:25:27.154680   
4   5        Panvel  18.9894  73.1175  2025-07-23 23:25:27.893369   

   current_speed  free_flow_speed  confidence road_class  is_weekend  \
0           27.0             27.0         1.0    Unknown           0   
1           22.0             22.0         1.0    Unknown           0   
2           31.0             31.0         1.0    Unknown           0   
3           23.0             23.0         1.0    Unknown           0   
4           21.0             21.0         1.0    Unknown           0   

   is_rush_hour  congestion_level  traffic_ratio  speed_drop_percent  
0             0               0.0            1.0               

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1146 entries, 0 to 1145
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  1146 non-null   int64         
 1   location_name       1146 non-null   object        
 2   lat                 1146 non-null   float64       
 3   lon                 1146 non-null   float64       
 4   timestamp           1146 non-null   datetime64[ns]
 5   current_speed       1146 non-null   float64       
 6   free_flow_speed     1146 non-null   float64       
 7   confidence          1146 non-null   float64       
 8   road_class          1146 non-null   object        
 9   is_weekend          1146 non-null   int64         
 10  is_rush_hour        1146 non-null   int64         
 11  congestion_level    1146 non-null   float64       
 12  traffic_ratio       1146 non-null   float64       
 13  speed_drop_percent  1146 non-null   float64     

In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [9]:
df.head

<bound method NDFrame.head of         id location_name      lat      lon                  timestamp  \
0        1         Dadar  19.0176  72.8562 2025-07-23 23:25:24.659867   
1        2       Andheri  19.1197  72.8468 2025-07-23 23:25:25.672796   
2        3         Powai  19.1197  72.9051 2025-07-23 23:25:26.528793   
3        4        Colaba  18.9067  72.8147 2025-07-23 23:25:27.154680   
4        5        Panvel  18.9894  73.1175 2025-07-23 23:25:27.893369   
...    ...           ...      ...      ...                        ...   
1141  1142        Panvel  18.9894  73.1175 2025-07-26 16:33:18.137767   
1142  1143        Airoli  19.1551  72.9960 2025-07-26 16:33:18.865597   
1143  1144         Vashi  19.0760  72.9986 2025-07-26 16:33:19.355936   
1144  1145         Nerul  19.0334  73.0186 2025-07-26 16:33:19.870760   
1145  1146      Kharghar  19.0330  73.0633 2025-07-26 16:33:20.327741   

      current_speed  free_flow_speed  confidence road_class  is_weekend  \
0              27.

In [10]:
df.describe()

,id,lat,lon,timestamp,current_speed,free_flow_speed,confidence,is_weekend,is_rush_hour,congestion_level,traffic_ratio,speed_drop_percent
count,1146.00000,1146.000000,1146.000000,1146,1146.000000,1146.000000,1146.000000,1146.000000,1146.000000,1146.000000,1146.000000,1146.000000
mean,573.50000,19.050160,72.957192,2025-07-25 06:39:52.283887616,19.950262,22.470332,0.976619,0.214660,0.094241,2.520070,0.892421,10.757871
min,1.00000,18.906700,72.814700,2025-07-23 23:25:24.659867,6.000000,14.000000,0.609692,0.000000,0.000000,0.000000,0.375000,0.000000
25%,287.25000,19.017600,72.856200,2025-07-24 14:58:58.699296256,17.000000,21.000000,0.985766,0.000000,0.000000,0.000000,0.772727,0.000000
50%,573.50000,19.033400,72.996000,2025-07-25 11:54:08.036718592,20.000000,22.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,859.75000,19.119700,73.018600,2025-07-25 22:51:39.171272448,22.000000,24.000000,1.000000,0.000000,0.000000,5.000000,1.000000,22.727273
max,1146.00000,19.155100,73.117500,2025-07-26 16:33:20.327741,31.000000,31.000000,1.000000,1.000000,1.000000,10.000000,1.000000,62.500000
std,330.96601,0.072466,0.099663,NaN,4.435089,4.362153,0.053245,0.410765,0.292291,3.071781,0.129363,12.936283


In [11]:
df['location_name'].value_counts()

location_name
Dadar       128
Andheri     128
Powai       128
Colaba      127
Panvel      127
Airoli      127
Vashi       127
Nerul       127
Kharghar    127
Name: count, dtype: int64

In [12]:
df['is_weekend'].value_counts()

is_weekend
0    900
1    246
Name: count, dtype: int64

In [13]:
df['is_rush_hour'].value_counts()

is_rush_hour
0    1038
1     108
Name: count, dtype: int64

In [15]:
df.duplicated().sum()

np.int64(0)

In [18]:
df.isnull().sum()

id                    0
location_name         0
lat                   0
lon                   0
timestamp             0
current_speed         0
free_flow_speed       0
confidence            0
road_class            0
is_weekend            0
is_rush_hour          0
congestion_level      0
traffic_ratio         0
speed_drop_percent    0
dtype: int64

In [20]:
df[(df['traffic_ratio'] > 1.2) & (df['traffic_ratio']< 0)]


,id,location_name,lat,lon,timestamp,current_speed,free_flow_speed,confidence,road_class,is_weekend,is_rush_hour,congestion_level,traffic_ratio,speed_drop_percent


In [21]:
df['location_name'].unique()


array(['Dadar', 'Andheri', 'Powai', 'Colaba', 'Panvel', 'Airoli', 'Vashi',
       'Nerul', 'Kharghar'], dtype=object)